<center>
<img src="https://upload.wikimedia.org/wikipedia/commons/a/a8/%D0%9B%D0%9E%D0%93%D0%9E_%D0%A8%D0%90%D0%94.png" width=300px/>
<br />
<h1>Дескрипторы и Метаклассы</h1>
<h3>Python</h3>
<br />
<h4>2018</h4> </center>

## Вспомним про  \_\_getattr__

In [2]:
class Test(object):
    a = 1
    def __getattr__(self, attr):
        print('Test::__getattr__(', attr, ')')
        return 42
    
print(Test().a)
print(Test().b) 

1
Test::__getattr__( b )
42


### \_\_getattribute__

In [3]:
class Test(object):
    a = 1
    def __getattribute__(self, attr):
        print('Test::__getattribute__(', attr, ')')
        return 

t = Test()
t.a

Test::__getattribute__( a )


In [6]:
class Test(object):
    a = 1
    def __getattribute__(self, attr):
        print('Test::__getattribute__(', attr, ')')
        return super().__getattribute__(attr)

t = Test()
t.a

Test::__getattribute__( a )


1

In [7]:
class Test(object):
    a = 1
    def __getattribute__(self, attr):
        print('Test::__getattribute__(', attr, ')')
        return object.__getattribute__(self, attr)
    
    def __getattr__(self, attr):
        print('Test::__getattr__(', attr, ')')
        return 42
    
t = Test()
t.b

Test::__getattribute__( b )
Test::__getattr__( b )


42

In [10]:
class Test(object):
    a = 1
    def __getattribute__(self, attr):
        print('Test::__getattribute__(', attr, ')')
        raise AttributeError()
       # return object.__getattribute__(self, attr)
    

    def __getattr__(self, attr):
#         raise AttributeError()
        return 42
    
t = Test()
t.a

Test::__getattribute__( a )


42

In [ ]:
class object:
    def __getattribute__(self, attr):
        if hasattr(self, attr):
            return getattr(self, attr)
        
        if hasattr(self, '__getattr__'):
            return self.__getattr__(attr)

In [16]:
p = Parrot()

p.voltage = 500

print(p.voltage)

p.voltage = 2000
    

500


ValueError: Danger! High voltage!

## Properties

In [12]:
class Parrot(object):
    def __init__(self):
        self.__voltage = 1000
    
    def get_voltage(self):
        return self.__voltage
    
    def set_voltage(self, value):
        if value > 1000:
            raise ValueError('Danger! High voltage!')
        self.__voltage = value
    
merlin = Parrot()
print(merlin.get_voltage())

try:
    merlin.set_voltage(2000)
except ValueError as e:
    print(e)

1000
Danger! High voltage!


In [15]:
class Parrot(object):
    def __init__(self):
        self._voltage = 1000
    def get_voltage(self):
        return self._voltage
    def set_voltage(self, value):
        if value > 1000:
            raise ValueError('Danger! High voltage!')
        self._voltage = value
    
    voltage = property(get_voltage, set_voltage)

p = Parrot()
p.voltage = 1000
p.voltage

1000

In [17]:
type(p.voltage), type(Parrot.voltage)

(int, property)

### More convenient

In [14]:
class Parrot(object):
    def __init__(self):
        self._voltage = 1000
    
    @property
    def voltage(self):
        return self._voltage
    
    @voltage.setter
    def voltage(self, value):
        if value > 1000:
            raise ValueError('Danger! High voltage!')
        self._voltage = value


p = Parrot()
p.voltage = 500
print(p.voltage)

500


In [5]:
class Person(object):
    def __init__(self, name, surname):
        self.name = name
        self.surname = surname
    
    @property
    def fullname(self):
        return '{name} {surname}'.format(**self.__dict__)
    

p = Person('John', 'Doe')
p.fullname

'John Doe'

In [11]:
class Parrot(object):
    def __init__(self):
        self._voltage = 1000
    
    @property
    def voltage(self):
        return self._voltage
    
    @voltage.setter
    def voltage(self, value):
        if value > 1000:
            raise ValueError('Danger! High voltage!')
        self._voltage = value


p = Parrot()
print(p.voltage, type(p.voltage), Parrot.voltage, type(Parrot.voltage))

1000 <class 'int'> <property object at 0x10d12aae8> <class 'property'>


### Properties get tedious

In [14]:
class Parrot(object):
    def __init__(self):
        self._voltage = 1000
        self._dead    = 0
    
    @property
    def voltage(self):
        return self._voltage
    
    @voltage.setter
    def voltage(self, value):
        if value > 1000:
            raise ValueError('Danger! High voltage!')
        self._voltage = value
    
    @property
    def dead(self):
        return self._dead
    
    @dead.setter
    def dead(self, value):
        if value not in [True, False]:
            raise ValueError('What did you even mean by that?')
        self._dead = value


p = Parrot()
p.voltage = 1000
p.dead  = 1

### Descriptors

In [2]:
class Parrot(object):
    voltage = Limited(1, 1000)
    dead    = Limited(0, 1)

In [3]:
class Limited(object):
    def __init__(self, lo, hi):
        pass  
    def __get__(self, instance, owner):
        pass
    def __set__(self, instance, value):
        pass

class Parrot(object):
    voltage = Limited(1, 1000)
    dead    = Limited(0, 1)

charlie = Parrot()
charlie.voltage = 1e2
charlie.voltage

In [2]:
class Trace(object):
    def __set__(self, *args):
        print('__set__', args)
    def __get__(self, *args):
        print('__get__', args)
        return 666
    def __delete__(self, *args):
        print('__delete__', args)

class Test(object):
    attr = Trace()
        
t = Test()
t.attr
t.attr = 1
Test.attr
del t.attr

__get__ (<__main__.Test object at 0x10b6c6e80>, <class '__main__.Test'>)
__set__ (<__main__.Test object at 0x10b6c6e80>, 1)
__get__ (None, <class '__main__.Test'>)
__delete__ (<__main__.Test object at 0x10b6c6e80>,)


In [28]:
class Limited(object):
    def __init__(self, lo, hi, label):
        self.lo = lo
        self.hi = hi
        self.label = label
        
    def __get__(self, instance, owner):
        if instance is None:
            return self
        return instance.__dict__.get(self.label)
    
    def __set__(self, instance, value):
        if not (self.lo <= value <= self.hi) :
            raise ValueError("Out of Bounds: %s" % value)
        instance.__dict__[self.label] = value
    
    def __delete__(self, instance):
        if self.label in instance.__dict__:
            del instance.__dict__[self.label]

class Parrot(object):
    voltage = Limited(1, 1000, 'voltage')
 
charlie = Parrot()
charlie.voltage = 1e2
print(charlie.voltage)
del charlie.voltage
print(charlie.voltage)

100.0
None


In [29]:
class Descriptor(object):
    def __init__(self, label):
        self.label = label
    def __get__(self, instance, owner):
#         print('__get__', instance, owner)
        return instance.__dict__.get(self.label)
    def __set__(self, instance, value):
#         print('__set__')
        instance.__dict__[self.label] = value

class Foo(object):
    x = Descriptor('y')
    
f = Foo()
f.x = 5
print(f.x)

f.y = 4
print(f.x)

5
4


### Non-data descriptors

In [30]:
class TestDescr(object):
    def __get__(self, inst, owner):
        return 42
    
class BrokenParrot(object):
    x = TestDescr()
    
    def set_x(self):
        self.x = 2
        
p = BrokenParrot()
p.x
p.set_x()
print(p.x, BrokenParrot.x)

2 42


### WeakKeyDictionary

In [3]:
from weakref import WeakKeyDictionary

class Limited(object):
    def __init__(self, lo, hi):
        self.lo = lo
        self.hi = hi
        self.data = WeakKeyDictionary()
        
    def __get__(self, instance, owner):
        return self.data.get(instance)
    
    def __set__(self, instance, value):
        if not (self.lo <= value <= self.hi) :
            raise ValueError("Out of Bounds: %s" % value)
        self.data[instance] = value
    
    def __delete__(self, instance):
        if instance in self.data:
            del self.data[instance]
        

class Parrot(object):
    voltage = Limited(1, 1000)

charlie = Parrot()
charlie.voltage = 1e2
print(charlie.voltage)

del charlie.voltage
print(charlie.voltage)

100.0
None


### CallbackProperty

In [91]:
class CallbackProperty(object):
    def __init__(self, default=None):
        self.data = WeakKeyDictionary()
        self.default = default
        self.callbacks = WeakKeyDictionary()
        
    def __get__(self, instance, owner):
        if instance is None:
            return self        
        return self.data.get(instance, self.default)
    
    def __set__(self, instance, value):
        for callback in self.callbacks.get(instance, []):
            callback(value)
        self.data[instance] = value
        
    def add_callback(self, instance, callback):
        if instance not in self.callbacks:
            self.callbacks[instance] = []
        self.callbacks[instance].append(callback)
        
class BankAccount(object):
    balance = CallbackProperty(0)
    
def low_balance_warning(value):
    if value < 100:
        print("You are now poor")
                
ba = BankAccount()
BankAccount.balance.add_callback(ba, low_balance_warning)

ba.balance = 5000
print("Balance is %s" % ba.balance)
ba.balance = 99
print("Balance is %s" % ba.balance)

Balance is 5000
You are now poor
Balance is 99


In [73]:
class Property(object):
    def __init__(self, fget=None, fset=None, fdel=None, doc=None):
        self.fget = fget
        self.fset = fset
        self.fdel = fdel
        self.__doc__ = doc
        
    def __get__(self, obj, objtype=None):
        if obj is None:
            return self
        
        if self.fget is None:
            raise AttributeError("unreadable attribute")
        return self.fget(obj)
    
    def __set__(self, obj, value):
        if self.fset is None:
            raise AttributeError("can't set attribute")
        
        self.fset(obj, value)
    def __delete__(self, obj):
        if self.fdel is None:
            raise AttributeError("can't delete attribute")
        
        self.fdel(obj)

    def getter(self, fget):
        return type(self)(fget, self.fset, self.fdel, self.__doc__)

    def setter(self, fset):
        return type(self)(self.fget, fset, self.fdel, self.__doc__)

    def deleter(self, fdel):
        return type(self)(self.fget, self.fset, fdel, self.__doc__)

### Метаклассы

In [33]:
class Cls(object): 
    pass

c = Cls()
type(c), type(Cls), type(type)

(__main__.Cls, type, type)

In [53]:
def class_factory(name, bases, **kwargs):
    return type(name, bases, kwargs)

F = class_factory('DeepThought', (object, ), ans=42)
f = F()
print(f.ans, f, type(f), type(F)) 

42 <__main__.DeepThought object at 0x10d1f1b38> <class '__main__.DeepThought'> <class 'type'>


In [77]:
class Meta(type):
    pass

def class_factory(name, bases, **kwargs):
    return Meta(name, bases, kwargs)

F = class_factory('DeepThought', (object, ), ans=42)
f = F()
print(f.ans, f, type(f), type(F))

42 <__main__.DeepThought object at 0x10d0f8e48> <class '__main__.DeepThought'> <class '__main__.Meta'>


In [43]:
class Meta(type):
    def __init__(cls, name, base, attrs):
        super().__init__(name, base, attrs)
        cls.hola = lambda self: 'qwerty'

def class_factory(name, bases, **kwargs):
    return Meta(name, bases, kwargs)
 
F = class_factory('DeepThought', (object, ), ans=42)
f = F()
print(f.ans, f, type(f), type(F))
print(f.hola())

42 <__main__.DeepThought object at 0x10eca70f0> <class '__main__.DeepThought'> <class '__main__.Meta'>
qwerty


In [5]:
class Meta(type):
    def __new__(mcs, name, bases, attrs, **kwargs):
        print('Meta.__new__(mcs=%s, name=%r, bases=%s, attrs=[%s])' % (mcs, name, bases, ', '.join(attrs)))
        return super().__new__(mcs, name, bases, attrs)
    def __init__(cls, name, bases, attrs):
        print('Meta.__init__(cls=%s, name=%r, bases=%s, attrs=[%s])' % (cls, name, bases, ', '.join(attrs)))
        return super().__init__(name, bases, attrs)

    def __call__(cls, *args, **kwargs):
        print('Meta.__call__(cls=%s, args=%s)' % (cls, args))
        return super().__call__(*args, **kwargs)

def class_factory(name, bases, **kwargs):
    return Meta(name, bases, kwargs)

F = class_factory('DeepThought', (object, ), ans=42)
print('*' * 20) 
f = F() 
print(f.ans, f, type(f), type(F))

Meta.__new__(mcs=<class '__main__.Meta'>, name='DeepThought', bases=(<class 'object'>,), attrs=[ans])
Meta.__init__(cls=<class '__main__.DeepThought'>, name='DeepThought', bases=(<class 'object'>,), attrs=[ans])
********************
Meta.__call__(cls=<class '__main__.DeepThought'>, args=())
42 <__main__.DeepThought object at 0x10b6c9588> <class '__main__.DeepThought'> <class '__main__.Meta'>


In [6]:
class Meta(type):
    def __init__(cls, name, base, attrs):
        super().__init__(name, base, attrs)
        cls.f = lambda self: 'qwerty'
        

class A(object, metaclass=Meta):
    pass

a = A()
print(a.f())

qwerty


In [12]:
class A(object, metaclass=Meta):
    pass

class B(A):
    pass

b = B()
[x for x in dir(B) if '__' not in x]

['f']

In [13]:
class M(type):
    def f(cls): 
        print('hello')
    
class Cls(object, metaclass=M):
    pass
    
inst = Cls()

Cls.f()
inst.f()

hello


AttributeError: 'Cls' object has no attribute 'f'

In [36]:
def upper_attr(future_class_name, future_class_parents, future_class_attr):
    uppercase_attr = {}
    for name, val in future_class_attr.items():
        if not name.startswith('__'):
            uppercase_attr[name.upper()] = val
        else:
            uppercase_attr[name] = val
    return type(future_class_name, future_class_parents, uppercase_attr)


class Foo(metaclass=upper_attr): 
    bip = 'bop'

f = Foo()
print(hasattr(Foo, 'bip'))
print(hasattr(Foo, 'BIP'))
print(f.BIP)

False
True
bop


In [138]:
class Descriptor(object):
    def __init__(self):
        #notice we aren't setting the label here
        self.label = None
        
    def __get__(self, instance, owner):
        print('__get__. Label = {}'.format(self.label))
        return instance.__dict__.get(self.label, None)
    
    def __set__(self, instance, value):
        print('__set__')
        instance.__dict__[self.label] = value

        
class DescriptorOwner(type):
    def __new__(cls, name, bases, attrs):
        for attr_name, attr_value in attrs.items():
            if isinstance(attr_value, Descriptor):
                attr_value.label = attr_name
        return super().__new__(cls, name, bases, attrs)

        
class Foo(object, metaclass=DescriptorOwner):
    x = Descriptor()
    
f = Foo()
f.x = 10
print(f.x)

__set__
__get__. Label = x
10
